In [5]:
import pandas as pd
import pickle

In [6]:
df = pd.read_csv('phil_nlp.csv')

df.sample(5)

,title,author,school,sentence_spacy,sentence_str,original_publication_date,corpus_edition_date,sentence_length,sentence_lowered,tokenized_txt,lemmatized_str
227957,The Crisis Of The European Sciences And Phenom...,Husserl,phenomenology,"It could only be inductive inference,.e., infe...","It could only be inductive inference,.e., infe...",1936,1970,92,"it could only be inductive inference,.e., infe...","['it', 'could', 'only', 'be', 'inductive', 'in...","-PRON- could only be inductive inference,.e ...."
39523,Aristotle - Complete Works,Aristotle,aristotle,"Further, if the conclusion is necessary, it fo...","Further, if the conclusion is necessary, it fo...",-320,1991,93,"further, if the conclusion is necessary, it fo...","['further', 'if', 'the', 'conclusion', 'is', '...","further , if the conclusion be necessary , -P..."
2214,Plato - Complete Works,Plato,plato,"Following Schofield, Classical Quarterly, we t...","Following Schofield, Classical Quarterly, we t...",-350,1997,64,"following schofield, classical quarterly, we t...","['following', 'schofield', 'classical', 'quart...","follow Schofield , Classical Quarterly , -PRO..."
363201,Confessions Of St. Augustine,Augustine,Scholasticism,"These things do I within, in that vast court o...","These things do I within, in that vast court o...",398,2002,58,NaN,"['these', 'things', 'do', 'within', 'in', 'tha...","these thing do -PRON- within , in that vast c..."
247274,Off The Beaten Track,Heidegger,phenomenology,He was able to do this through the God given a...,He was able to do this through the God given a...,1950,2001,55,he was able to do this through the god given a...,"['he', 'was', 'able', 'to', 'do', 'this', 'thr...",-PRON- be able to do this through the God giv...


In [7]:
for_db = df
for_db['date'] = for_db['original_publication_date']
for_db['date'] = for_db['date'].apply(lambda x: str(x)[1:]+' BC' if x < 0 else str(x))
for_db['sentence'] = for_db['sentence_str']
for_db['school'] = for_db['school'].apply(lambda x: x.replace('_', ' ').title())
for_db = for_db.drop(['sentence_spacy', 
                      'sentence_length',
                      'sentence_lowered', 
                      'sentence_str', 
                      'tokenized_txt', 
                      'lemmatized_str',
                      'corpus_edition_date',
                      'original_publication_date'], axis=1)
for_db.columns = [i.upper() for i in for_db.columns]

for_db.sample(5)


,TITLE,AUTHOR,SCHOOL,DATE,SENTENCE
344264,Vindication Of The Rights Of Woman,Wollstonecraft,Feminism,1792,"His father's house was his home, and was ever ..."
295133,Capital,Marx,Communism,1883,"If one hour's labour is embodied in sixpence, ..."
253261,Critique Of Judgement,Kant,German Idealism,1790,"But, where in man, at any rate, are we to plac..."
220301,The Phenomenology Of Perception,Merleau-Ponty,Phenomenology,1945,"Sense experience, on the other hand, invests t..."
342723,Vindication Of The Rights Of Woman,Wollstonecraft,Feminism,1792,What is then to become of her?


In [14]:
for_db.sample(5)

,TITLE,AUTHOR,SCHOOL,YEAR,SENTENCE
139007,Philosophical Investigations,Wittgenstein,Analytic,1953,To repeat naming is something like attaching a...
226005,The Phenomenology Of Perception,Merleau-Ponty,Phenomenology,1945,"that of the world, this is because my existenc..."
129412,The Search After Truth,Malebranche,Rationalism,1674,Since rays consist only in rectilinear pressur...
355456,The Second Sex,Beauvoir,Feminism,1949,This is what she anxiously wonders while she w...
351961,The Second Sex,Beauvoir,Feminism,1949,"At she told herself, 'I'll be a lieutenant, an..."


In [8]:
for_db['SCHOOL'].value_counts()

Analytic           55425
Aristotle          48779
German Idealism    42136
Plato              38366
Continental        33779
Phenomenology      28573
Rationalism        22949
Empiricism         19931
Feminism           18635
Capitalism         18194
Communism          17958
Nietzsche          13548
Scholasticism       4068
Stoicism            2535
Name: SCHOOL, dtype: int64

In [11]:
classifier_dict = {}
for author in for_db['AUTHOR'].unique():
  classifier_dict[author] = 'AUTHOR'
for title in for_db['TITLE'].unique():
  classifier_dict[title] = 'TITLE'
for school in for_db['SCHOOL'].unique():
  classifier_dict[school] = 'SCHOOL'

In [12]:
classifier_dict

{'Plato': 'SCHOOL',
 'Aristotle': 'SCHOOL',
 'Locke': 'AUTHOR',
 'Hume': 'AUTHOR',
 'Berkeley': 'AUTHOR',
 'Spinoza': 'AUTHOR',
 'Leibniz': 'AUTHOR',
 'Descartes': 'AUTHOR',
 'Malebranche': 'AUTHOR',
 'Russell': 'AUTHOR',
 'Moore': 'AUTHOR',
 'Wittgenstein': 'AUTHOR',
 'Lewis': 'AUTHOR',
 'Quine': 'AUTHOR',
 'Popper': 'AUTHOR',
 'Kripke': 'AUTHOR',
 'Foucault': 'AUTHOR',
 'Derrida': 'AUTHOR',
 'Deleuze': 'AUTHOR',
 'Merleau-Ponty': 'AUTHOR',
 'Husserl': 'AUTHOR',
 'Heidegger': 'AUTHOR',
 'Kant': 'AUTHOR',
 'Fichte': 'AUTHOR',
 'Hegel': 'AUTHOR',
 'Marx': 'AUTHOR',
 'Lenin': 'AUTHOR',
 'Smith': 'AUTHOR',
 'Ricardo': 'AUTHOR',
 'Keynes': 'AUTHOR',
 'Epictetus': 'AUTHOR',
 'Marcus Aurelius': 'AUTHOR',
 'Nietzsche': 'SCHOOL',
 'Wollstonecraft': 'AUTHOR',
 'Beauvoir': 'AUTHOR',
 'Davis': 'AUTHOR',
 'Augustine': 'AUTHOR',
 'Plato - Complete Works': 'TITLE',
 'Aristotle - Complete Works': 'TITLE',
 'Second Treatise On Government': 'TITLE',
 'Essay Concerning Human Understanding': 'TITLE',
 'A

Now get an updated dropdown menu.

In [13]:
all_options = sorted([x for x in list(classifier_dict.keys())])

In [14]:
dropdown_menu = []
for source in all_options:
    dropdown_menu.append({'label': source, 'value': source})

dropdown_menu

[{'label': 'A General Theory Of Employment, Interest, And Money',
  'value': 'A General Theory Of Employment, Interest, And Money'},
 {'label': 'A Treatise Concerning The Principles Of Human Knowledge',
  'value': 'A Treatise Concerning The Principles Of Human Knowledge'},
 {'label': 'A Treatise Of Human Nature',
  'value': 'A Treatise Of Human Nature'},
 {'label': 'Analytic', 'value': 'Analytic'},
 {'label': 'Anti-Oedipus', 'value': 'Anti-Oedipus'},
 {'label': 'Aristotle', 'value': 'Aristotle'},
 {'label': 'Aristotle - Complete Works',
  'value': 'Aristotle - Complete Works'},
 {'label': 'Augustine', 'value': 'Augustine'},
 {'label': 'Beauvoir', 'value': 'Beauvoir'},
 {'label': 'Being And Time', 'value': 'Being And Time'},
 {'label': 'Berkeley', 'value': 'Berkeley'},
 {'label': 'Beyond Good And Evil', 'value': 'Beyond Good And Evil'},
 {'label': 'Capital', 'value': 'Capital'},
 {'label': 'Capitalism', 'value': 'Capitalism'},
 {'label': 'Communism', 'value': 'Communism'},
 {'label': 'C

In [25]:
for source in classifier_dict.keys():
        source_slice = df[df[classifier_dict[source]]==source]
        source = source.title()
        source_pkl = open(f'../search_app/slice_pickles/{source}_slice.pkl', 'wb')
        pickle.dump(source_slice, source_pkl)
        source_pkl.close()


FileNotFoundError: [Errno 2] No such file or directory: '../search_app/slice_pickles/Plato_slice.pkl'